# Assessing analysts accuracy at labelling reference data

Collect Earth Online is being used as a tool for collecting cropland reference data.  The sample data contains 'known' labels seeded among the other samples. This script will compare the known test labels (GFSAD's validation data), against the user collected labels.

Inputs will be:

1. `ceo-data....csv` : The results from collecting training data in the CEO tool

Output will be:
1. A `confusion error matrix` containing Overall, Producer's, and User's accuracy, along with the F1 score.

In [13]:
import pandas as pd
import numpy as np
import seaborn as sn
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

## Analysis Parameters

In [4]:
csv = 'data/training_validation/collect_earth/testing_CEO_results/ceo-cropland-training-&-validation-data-acquistion---eastern-region-sample-data-2020-08-26.csv'


### Load the dataset

In [5]:
#ground truth shapefile
df = pd.read_csv(csv)

### Clean up dataframe

In [10]:
#only grab useful columns
df = df[['LON', 'LAT', 'SMPL_SAMPLEID', 'SMPL_GFSAD_SAMP','SMPL_CLASS','IS THE SAMPLE AREA ENTIRELY: CROP, NON-CROP, MIXED, OR UNSURE?']]

#rename columns
df = df.rename(columns={'IS THE SAMPLE AREA ENTIRELY: CROP, NON-CROP, MIXED, OR UNSURE?':'Prediction',
                        'SMPL_GFSAD_CLASS':'Actual'})

#remove nan rows
df = df.dropna()
df.head()

,LON,LAT,SMPL_SAMPLEID,SMPL_GFSAD_SAMP,SMPL_CLASS,Prediction
1,38.525059,12.278173,42,0,1,non-crop
16,35.274416,5.608182,233,0,1,non-crop
19,38.971342,-1.848194,0,0,1,non-crop
26,38.304613,0.326088,109,0,1,non-crop
31,41.330498,3.890963,61,0,1,non-crop


### Generate a confusion matrix wih all classes

In [5]:
confusion_matrix = pd.crosstab(df['Actual'],
                               df['Prediction'],
                               rownames=['Actual'],
                               colnames=['Prediction'],
                               margins=True)

confusion_matrix

Prediction,0,1,2,3,All
Actual,,,,,
0,198,5,10,4,217
1,6,60,7,1,74
All,204,65,17,5,291


### Reclassify into a binary assessment

In [6]:
counts = df.groupby('Prediction').count()

print("Total number of samples: " + str(len(df)))
print("Number of 'mixed' samples: "+ str(counts[counts.index==2]['Actual'].values[0]))
print("Number of 'unsure' samples: "+ str(counts[counts.index==3]['Actual'].values[0]))

print("Reclassifying 'mixed' and 'unsure' sample to 'non-crop' ")

df['Prediction'] = np.where(df['Prediction']==2, 0, df['Prediction'])
df['Prediction'] = np.where(df['Prediction']==3, 0, df['Prediction'])

Total number of samples: 291
Number of 'mixed' samples: 17
Number of 'unsure' samples: 5
Reclassifying 'mixed' and 'unsure' sample to 'non-crop' 


---

### Recreate confusion matrix

In [7]:
confusion_matrix = pd.crosstab(df['Actual'],
                               df['Prediction'],
                               rownames=['Actual'],
                               colnames=['Prediction'],
                               margins=True)

confusion_matrix

Prediction,0,1,All
Actual,,,
0,212,5,217
1,14,60,74
All,226,65,291


### Calculate User's and Producer's Accuracy

`User's Accuracy`

In [8]:
confusion_matrix["User's"] = [confusion_matrix.loc[0, 0] / confusion_matrix.loc[0, 'All'] * 100,
                              confusion_matrix.loc[1, 1] / confusion_matrix.loc[1, 'All'] * 100,
                              np.nan]

`Producer's Accuracy`

In [9]:
producers_accuracy = pd.Series([confusion_matrix[0][0] / confusion_matrix[0]['All'] * 100,
                                confusion_matrix[1][1] / confusion_matrix[1]['All'] * 100]
                         ).rename("Producer's")

confusion_matrix = confusion_matrix.append(producers_accuracy)

`Overall Accuracy`

In [10]:
confusion_matrix.loc["Producer's", "User's"] = (confusion_matrix.loc[0, 0] + 
                                                confusion_matrix.loc[1, 1]) / confusion_matrix.loc['All', 'All'] * 100

`F1 Score`

The F1 score is the harmonic mean of the precision and recall, where an F1 score reaches its best value at 1 (perfect precision and recall), and is calculated as:

$$
\begin{aligned}
\text{Fscore} = 2 \times \frac{\text{UA} \times \text{PA}}{\text{UA} + \text{PA}}.
\end{aligned}
$$

Where UA = Users Accuracy, and PA = Producer's Accuracy

In [11]:
fscore = pd.Series([(2*(confusion_matrix.loc[0, "User's"]*confusion_matrix.loc["Producer's", 0]) / (confusion_matrix.loc[0, "User's"]+confusion_matrix.loc["Producer's", 0])) / 100,
                    f1_score(df['Actual'].astype(np.int8), df['Prediction'].astype(np.int8), average='binary')]
                         ).rename("F-score")

confusion_matrix = confusion_matrix.append(fscore)

### Tidy Confusion Matrix

* Limit decimal places,
* Add readable class names
* Remove non-sensical values 

In [12]:
# round numbers
confusion_matrix = confusion_matrix.round(decimals=2)

In [13]:
# rename booleans to class names
confusion_matrix = confusion_matrix.rename(columns={0:'Non-crop', 1:'Crop', 'All':'Total'},
                                            index={0:'Non-crop', 1:'Crop', 'All':'Total'})

In [14]:
#remove the nonsensical values in the table
confusion_matrix.loc['Total', "User's"] = '--'
confusion_matrix.loc["Producer's", 'Total'] = '--'
confusion_matrix.loc["F-score", 'Total'] = '--'
confusion_matrix.loc["F-score", "User's"] = '--'

In [15]:
confusion_matrix

Prediction,Non-crop,Crop,Total,User's
Actual,,,,
Non-crop,212.00,5.00,217,97.7
Crop,14.00,60.00,74,81.08
Total,226.00,65.00,291,--
Producer's,93.81,92.31,--,93.47
F-score,0.96,0.86,--,--


### Export csv

In [ ]:
# confusion_matrix.to_csv('results/confusion_matrix.csv')